In [ ]:
%pip install pandas
%pip install scikit-learn
%pip install numpy
%pip install matplotlib
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

df = pd.read_csv("/Users/admin/AMR/data/cleaned/done.csv")






In [3]:
df.shape

(12045, 5)

In [4]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')

In [5]:
df.dtypes

geography                 object
stratum                   object
date              datetime64[ns]
organism                  object
resistance_pct           float64
dtype: object

In [6]:
print(df['date'])


0       2020-10-01
1       2020-11-01
2       2020-12-01
3       2021-01-01
4       2021-02-01
           ...    
12040   2025-02-01
12041   2025-03-01
12042   2025-04-01
12043   2025-05-01
12044   2025-06-01
Name: date, Length: 12045, dtype: datetime64[ns]


In [7]:
df['date'].isna().sum()

np.int64(0)

In [8]:
df = df.sort_values(
    ["geography", "stratum", "organism", "date"]
)

In [9]:
df["geography"].value_counts()

geography
East of England         1339
West Midlands           1339
London                  1339
North West              1339
South East              1339
Yorkshire and Humber    1339
South West              1339
East Midlands           1337
North East              1335
Name: count, dtype: int64

In [10]:
df.groupby(
    ["geography", "stratum", "organism"]
)["date"].count().describe()

count    207.000000
mean      58.188406
std        0.979529
min       57.000000
25%       57.000000
50%       59.000000
75%       59.000000
max       59.000000
Name: date, dtype: float64

Forecast horizon - 6 months
Window Length - 12 months
Static features - (Organism,Antibiotic, region)
Dynamic features - Resistance history over time

Baseline/Benchmark - Linear regression

In [11]:
df["series_id"] = (
    df["organism"].astype(str)
    + "__" + df["stratum"].astype(str)
    + "__" + df["geography"].astype(str)
)

In [12]:
df = df.sort_values(["series_id", "date"])

In [13]:
df["series_id"].tail()

12040    e-faecium__Glycopeptides__Yorkshire and Humber
12041    e-faecium__Glycopeptides__Yorkshire and Humber
12042    e-faecium__Glycopeptides__Yorkshire and Humber
12043    e-faecium__Glycopeptides__Yorkshire and Humber
12044    e-faecium__Glycopeptides__Yorkshire and Humber
Name: series_id, dtype: object

In [14]:
df.head()

,geography,stratum,date,organism,resistance_pct,series_id
0,East Midlands,Amikacin,2020-10-01,E-coli,0.53,E-coli__Amikacin__East Midlands
1,East Midlands,Amikacin,2020-11-01,E-coli,0.92,E-coli__Amikacin__East Midlands
2,East Midlands,Amikacin,2020-12-01,E-coli,0.93,E-coli__Amikacin__East Midlands
3,East Midlands,Amikacin,2021-01-01,E-coli,0.53,E-coli__Amikacin__East Midlands
4,East Midlands,Amikacin,2021-02-01,E-coli,0.17,E-coli__Amikacin__East Midlands


In [15]:
y = (
    df.set_index(["series_id", "date"])
    ["resistance_pct"]
)

In [16]:
region_map = {
    "East Midlands": "Midlands",
    "West Midlands": "Midlands",
    "North East": "North East and Yorkshire",
    "Yorkshire and Humber": "North East and Yorkshire"
}

In [17]:
df["model_region"] = df["geography"].replace(region_map)


In [18]:
import glob
import os
base_path = "/Users/admin/AMR/data/raw/prescribingdata"
files = glob.glob(os.path.join(base_path, "*.csv"))

Prescribing Data Cleaning

In [19]:
files

['/Users/admin/AMR/data/raw/prescribingdata/eastofengland.csv',
 '/Users/admin/AMR/data/raw/prescribingdata/midlands.csv',
 '/Users/admin/AMR/data/raw/prescribingdata/south west.csv',
 '/Users/admin/AMR/data/raw/prescribingdata/south east.csv',
 '/Users/admin/AMR/data/raw/prescribingdata/north west.csv',
 '/Users/admin/AMR/data/raw/prescribingdata/north east and york.csv',
 '/Users/admin/AMR/data/raw/prescribingdata/london.csv']

In [20]:

prescribing_list = []
for f in files :
    file = pd.read_csv(f)
    prescribing_list.append(file)
prescribing = pd.concat(prescribing_list, ignore_index=True)

In [21]:
prescribing["Area Type"].value_counts()
#prescribing["Area Name"].value_counts().head(20)


Area Type
ICB sub-locations    19758
England               1494
NHS regions           1482
Name: count, dtype: int64

In [22]:
prescribing_nhs = prescribing[
    prescribing["Area Type"] == "NHS regions"
].copy()

In [23]:
prescribing_nhs["Area Name"].unique()

array(['East of England NHS Region', 'Midlands NHS Region',
       'South West NHS Region', 'South East NHS Region',
       'North West NHS Region', 'North East and Yorkshire NHS Region',
       'London NHS Region'], dtype=object)

In [24]:
prescribing_region_map = {
    "Midlands NHS Region": "Midlands",
    "North East and Yorkshire NHS region" : "North East and Yorkshire",
    "South West NHS Region": "South West",
    "South East NHS Region": "South East",
    "London NHS Region": "London",
    "East of England NHS Region": "East of England",
    "North West NHS Region": "North West"
}


In [25]:
prescribing_nhs["model_region"] = prescribing_nhs["Area Name"].replace(
    prescribing_region_map
)

In [26]:
prescribing_nhs["model_region"].isna().sum()

np.int64(0)

In [27]:
prescribing_nhs.head()

,Indicator ID,Indicator Name,Parent Code,Parent Name,Area Code,Area Name,Area Type,Sex,Age,Category Type,...,Denominator,Value note,Recent Trend,Compared to England value or percentiles,Compared to NHS regions value or percentiles,Time period Sortable,New data,Compared to goal,Time period range,model_region
1,92344,Twelve-month rolling total number of prescribe...,E92000001,England,E40000007,East of England NHS Region,NHS regions,Persons,All ages,NaN,...,3536917.0,Aggregated from all known lower geography values,NaN,Not compared,Not compared,20140001,NaN,NaN,1m,East of England
14,92344,Twelve-month rolling total number of prescribe...,E92000001,England,E40000007,East of England NHS Region,NHS regions,Persons,All ages,NaN,...,3536917.0,Aggregated from all known lower geography values,NaN,Not compared,Not compared,20140002,NaN,NaN,1m,East of England
27,92344,Twelve-month rolling total number of prescribe...,E92000001,England,E40000007,East of England NHS Region,NHS regions,Persons,All ages,NaN,...,3536917.0,Aggregated from all known lower geography values,NaN,Not compared,Not compared,20140003,NaN,NaN,1m,East of England
40,92344,Twelve-month rolling total number of prescribe...,E92000001,England,E40000007,East of England NHS Region,NHS regions,Persons,All ages,NaN,...,3536917.0,Aggregated from all known lower geography values,NaN,Not compared,Not compared,20140004,NaN,NaN,1m,East of England
53,92344,Twelve-month rolling total number of prescribe...,E92000001,England,E40000007,East of England NHS Region,NHS regions,Persons,All ages,NaN,...,3536917.0,Aggregated from all known lower geography values,NaN,Not compared,Not compared,20140005,NaN,NaN,1m,East of England


In [28]:
prescribing_nhs["Time period"].head(10)


1      Jan-14
14     Feb-14
27     Mar-14
40     Apr-14
53     May-14
66     Jun-14
79     Jul-14
92     Aug-14
105    Sep-14
118    Oct-14
Name: Time period, dtype: object

In [29]:
prescribing_nhs["Time period"].value_counts().head(10)


Time period
May-15    10
Jun-15    10
Jul-15    10
Aug-15    10
Jan-15    10
Feb-15    10
Mar-15    10
Apr-15    10
Sep-15    10
Oct-15    10
Name: count, dtype: int64

In [30]:
monthly = prescribing_nhs[
    prescribing_nhs["Time period"].str.match(r"^[A-Za-z]{3}-\d{2}$", na=False)
].copy()


In [31]:
monthly["date"] = pd.to_datetime(
    monthly["Time period"],
    format="%b-%y"
)


In [32]:
prescribing_clean = monthly[
    [
        "Area Name",
        "date",
        "Value"
    ]
].rename(columns={
    "Area Name": "model_region",
    "Value": "prescribing_rate"
})


In [33]:
prescribing_clean.dtypes

model_region                object
date                datetime64[ns]
prescribing_rate           float64
dtype: object

In [34]:
print("Prescribing regions: ")
print(prescribing_clean["model_region"].unique())

print("\nAMR regions: ")
print(df["model_region"].unique())

Prescribing regions: 
['East of England NHS Region' 'Midlands NHS Region'
 'South West NHS Region' 'South East NHS Region' 'North West NHS Region'
 'North East and Yorkshire NHS Region' 'London NHS Region']

AMR regions: 
['Midlands' 'East of England' 'London' 'North East and Yorkshire'
 'North West' 'South East' 'South West']


In [35]:
df.head()

,geography,stratum,date,organism,resistance_pct,series_id,model_region
0,East Midlands,Amikacin,2020-10-01,E-coli,0.53,E-coli__Amikacin__East Midlands,Midlands
1,East Midlands,Amikacin,2020-11-01,E-coli,0.92,E-coli__Amikacin__East Midlands,Midlands
2,East Midlands,Amikacin,2020-12-01,E-coli,0.93,E-coli__Amikacin__East Midlands,Midlands
3,East Midlands,Amikacin,2021-01-01,E-coli,0.53,E-coli__Amikacin__East Midlands,Midlands
4,East Midlands,Amikacin,2021-02-01,E-coli,0.17,E-coli__Amikacin__East Midlands,Midlands


In [36]:
prescribing_clean.head()

,model_region,date,prescribing_rate
1,East of England NHS Region,2014-01-01,1.925150
14,East of England NHS Region,2014-02-01,1.916657
27,East of England NHS Region,2014-03-01,1.919446
40,East of England NHS Region,2014-04-01,1.918850
53,East of England NHS Region,2014-05-01,1.924075


In [37]:
region_map = {
    "East of England NHS Region": "East of England",
    "Midlands NHS Region":"Midlands",
    "London NHS Region": "London",
    "North East and Yorkshire NHS Region":"North East and Yorkshire",
    "North West NHS Region": "North West",
    "South East NHS Region": "South East",
    "South West NHS Region": "South West"
}

In [38]:
prescribing_clean["region"] = (
    prescribing_clean["model_region"]
    .map(region_map)
)

In [39]:
prescribing_clean.head()

,model_region,date,prescribing_rate,region
1,East of England NHS Region,2014-01-01,1.925150,East of England
14,East of England NHS Region,2014-02-01,1.916657,East of England
27,East of England NHS Region,2014-03-01,1.919446,East of England
40,East of England NHS Region,2014-04-01,1.918850,East of England
53,East of England NHS Region,2014-05-01,1.924075,East of England


In [40]:
prescribing_clean["region"].value_counts(dropna=False)

region
South East                  528
Midlands                    138
East of England             138
South West                  138
North West                  138
North East and Yorkshire    138
London                      126
Name: count, dtype: int64

In [41]:
prescribing_clean = prescribing_clean.rename(
    columns={"region": "model_region"}
)

In [42]:

prescribing_clean.columns = [
    "model_region_nhs",
    "date",
    "prescribing_rate",
    "model_region"
]


In [43]:
prescribing_clean = prescribing_clean.drop(columns=["model_region_nhs"])

In [44]:
prescribing_clean.head()

,date,prescribing_rate,model_region
1,2014-01-01,1.925150,East of England
14,2014-02-01,1.916657,East of England
27,2014-03-01,1.919446,East of England
40,2014-04-01,1.918850,East of England
53,2014-05-01,1.924075,East of England


In [45]:
df.info

<bound method DataFrame.info of                   geography        stratum       date   organism  \
0             East Midlands       Amikacin 2020-10-01     E-coli   
1             East Midlands       Amikacin 2020-11-01     E-coli   
2             East Midlands       Amikacin 2020-12-01     E-coli   
3             East Midlands       Amikacin 2021-01-01     E-coli   
4             East Midlands       Amikacin 2021-02-01     E-coli   
...                     ...            ...        ...        ...   
12040  Yorkshire and Humber  Glycopeptides 2025-02-01  e-faecium   
12041  Yorkshire and Humber  Glycopeptides 2025-03-01  e-faecium   
12042  Yorkshire and Humber  Glycopeptides 2025-04-01  e-faecium   
12043  Yorkshire and Humber  Glycopeptides 2025-05-01  e-faecium   
12044  Yorkshire and Humber  Glycopeptides 2025-06-01  e-faecium   

       resistance_pct                                       series_id  \
0                0.53                 E-coli__Amikacin__East Midlands   
1    

Merge AMR and Prescribing Datasets

In [46]:
prescribing_clean.duplicated(
    ["model_region", "date"]
).sum()


np.int64(390)

In [47]:
prescribing_clean.to_csv("prescribingdata.csv")


In [48]:
df = df.sort_values(["model_region", "date"])
prescribing_clean = prescribing_clean.sort_values(["model_region", "date"])

combined_df = df.merge(
    prescribing_clean,
    on=["model_region", "date"],
    how="left"
)


In [49]:
combined_df.info

<bound method DataFrame.info of              geography                       stratum       date      organism  \
0      East of England                      Amikacin 2020-08-01  K-pneumoniae   
1      East of England                   Carbapenems 2020-08-01  K-pneumoniae   
2      East of England                 Ciprofloxacin 2020-08-01  K-pneumoniae   
3      East of England                  Co-amoxiclav 2020-08-01  K-pneumoniae   
4      East of England                    Gentamicin 2020-08-01  K-pneumoniae   
...                ...                           ...        ...           ...   
16057       South West  Piperacillin with tazobactam 2025-06-01  P-aeruginosa   
16058       South West                    Macrolides 2025-06-01  S-pneumoniae   
16059       South West                    Penicillin 2025-06-01  S-pneumoniae   
16060       South West                 Glycopeptides 2025-06-01    e-faecalis   
16061       South West                 Glycopeptides 2025-06-01     e-faecium

In [50]:
combined_df.head()

,geography,stratum,date,organism,resistance_pct,series_id,model_region,prescribing_rate
0,East of England,Amikacin,2020-08-01,K-pneumoniae,0.64,K-pneumoniae__Amikacin__East of England,East of England,1.52
1,East of England,Carbapenems,2020-08-01,K-pneumoniae,0.00,K-pneumoniae__Carbapenems__East of England,East of England,1.52
2,East of England,Ciprofloxacin,2020-08-01,K-pneumoniae,11.06,K-pneumoniae__Ciprofloxacin__East of England,East of England,1.52
3,East of England,Co-amoxiclav,2020-08-01,K-pneumoniae,27.04,K-pneumoniae__Co-amoxiclav__East of England,East of England,1.52
4,East of England,Gentamicin,2020-08-01,K-pneumoniae,4.90,K-pneumoniae__Gentamicin__East of England,East of England,1.52


In [51]:
combined_df[["prescribing_rate"]].isna().sum()

prescribing_rate    0
dtype: int64

In [52]:
combined_df["prescribing_rate"].isna().mean()


np.float64(0.0)

Predict 1-6 month ahead antimicrobial resistance trends per bacterium-antibiotic-region combination

Import Time series library - sktime

In [53]:
%pip install sktime

Note: you may need to restart the kernel to use updated packages.


In [54]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [55]:
%pip install lightgbm


Note: you may need to restart the kernel to use updated packages.


In [56]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [57]:
from sklearn.ensemble import RandomForestRegressor

In [58]:
combined_df['date']= pd.to_datetime(combined_df['date'])

In [59]:
#sort by series_id and date to ensure temporal order within each series
combined_df = combined_df.sort_values(['series_id', 'date']).reset_index(drop=True)

In [60]:
combined_df['month'] = combined_df['date'].dt.month

In [61]:
combined_df.head()

,geography,stratum,date,organism,resistance_pct,series_id,model_region,prescribing_rate,month
0,East Midlands,Amikacin,2020-10-01,E-coli,0.53,E-coli__Amikacin__East Midlands,Midlands,1.46,10
1,East Midlands,Amikacin,2020-11-01,E-coli,0.92,E-coli__Amikacin__East Midlands,Midlands,1.43,11
2,East Midlands,Amikacin,2020-12-01,E-coli,0.93,E-coli__Amikacin__East Midlands,Midlands,1.38,12
3,East Midlands,Amikacin,2021-01-01,E-coli,0.53,E-coli__Amikacin__East Midlands,Midlands,1.33,1
4,East Midlands,Amikacin,2021-02-01,E-coli,0.17,E-coli__Amikacin__East Midlands,Midlands,1.29,2


Create Lags

In [62]:
window_length = 12

In [63]:
for lag in range(1, window_length + 1):
    combined_df[f'lag_{lag}'] = combined_df.groupby('series_id')['resistance_pct'].shift(lag)

In [64]:

for lag in [1, 3, 6]:
    combined_df[f'presc_lag_{lag}'] = (
        combined_df.groupby('series_id')['prescribing_rate']
                   .shift(lag)
    )

In [65]:
# Trend feature
combined_df['trend'] = combined_df.groupby('series_id')['resistance_pct'] \
    .transform(lambda x: x.shift(1) - x.shift(3))

# Interaction: prescribing × month
combined_df['prescribing_x_month'] = combined_df['prescribing_rate'] * combined_df['month']


In [66]:
combined_df.info

<bound method DataFrame.info of                   geography        stratum       date   organism  \
0             East Midlands       Amikacin 2020-10-01     E-coli   
1             East Midlands       Amikacin 2020-11-01     E-coli   
2             East Midlands       Amikacin 2020-12-01     E-coli   
3             East Midlands       Amikacin 2021-01-01     E-coli   
4             East Midlands       Amikacin 2021-02-01     E-coli   
...                     ...            ...        ...        ...   
16057  Yorkshire and Humber  Glycopeptides 2025-02-01  e-faecium   
16058  Yorkshire and Humber  Glycopeptides 2025-03-01  e-faecium   
16059  Yorkshire and Humber  Glycopeptides 2025-04-01  e-faecium   
16060  Yorkshire and Humber  Glycopeptides 2025-05-01  e-faecium   
16061  Yorkshire and Humber  Glycopeptides 2025-06-01  e-faecium   

       resistance_pct                                       series_id  \
0                0.53                 E-coli__Amikacin__East Midlands   
1    

In [67]:
combined_df = combined_df.dropna()

In [68]:
combined_df.info

<bound method DataFrame.info of                   geography        stratum       date   organism  \
12            East Midlands       Amikacin 2021-10-01     E-coli   
13            East Midlands       Amikacin 2021-11-01     E-coli   
14            East Midlands       Amikacin 2021-12-01     E-coli   
15            East Midlands       Amikacin 2022-01-01     E-coli   
16            East Midlands       Amikacin 2022-02-01     E-coli   
...                     ...            ...        ...        ...   
16057  Yorkshire and Humber  Glycopeptides 2025-02-01  e-faecium   
16058  Yorkshire and Humber  Glycopeptides 2025-03-01  e-faecium   
16059  Yorkshire and Humber  Glycopeptides 2025-04-01  e-faecium   
16060  Yorkshire and Humber  Glycopeptides 2025-05-01  e-faecium   
16061  Yorkshire and Humber  Glycopeptides 2025-06-01  e-faecium   

       resistance_pct                                       series_id  \
12               1.52                 E-coli__Amikacin__East Midlands   
13   

In [69]:
combined_df['date'].min(), combined_df['date'].max()


(Timestamp('2020-11-01 00:00:00'), Timestamp('2025-06-01 00:00:00'))

In [70]:
combined_df.groupby('series_id')['date'].min().sort_values().head(10)


series_id
K-pneumoniae__Piperacillin with tazobactam__South East      2020-11-01
K-pneumoniae__Third Generation Cephalosporins__South East   2020-11-01
K-pneumoniae__Carbapenems__South East                       2020-11-01
P-aeruginosa__Piperacillin with tazobactam__South East      2020-11-01
K-pneumoniae__Ciprofloxacin__South East                     2020-11-01
S-pneumoniae__Macrolides__South East                        2020-11-01
S-pneumoniae__Penicillin__South East                        2020-11-01
P-aeruginosa__Carbapenems__South East                       2020-11-01
P-aeruginosa__Ceftazidime__South East                       2020-11-01
P-aeruginosa__Amikacin__South East                          2020-11-01
Name: date, dtype: datetime64[ns]

In [71]:
print(f"Data before creating lags: {len(combined_df)} rows")
combined_df = combined_df.dropna()
print(f"Data after creating lags: {len(combined_df)} rows")

Data before creating lags: 13578 rows
Data after creating lags: 13578 rows


In [72]:
combined_df.to_csv("datasetslagfortraining.csv")

In [73]:
train_df = combined_df[combined_df['date']< '2024-03-01'].copy()
test_df = combined_df[combined_df['date'] >= '2024-03-01'].copy()

In [74]:
print(f"\nTrain: {len(train_df)} rows")
print(f"\nTest: {len(test_df)} rows")


Train: 9162 rows

Test: 4416 rows


In [75]:
combined_df['prescribing_x_month'] = combined_df['prescribing_rate'] * combined_df['month']


In [76]:
feature_cols = ([f'lag_{i}' for i in range (1, window_length +1)] + [f'presc_lag_{i}'for i in [1,3, 6]] + ['month'] + ['trend'] + ['prescribing_x_month'])

In [77]:
feature_cols

['lag_1',
 'lag_2',
 'lag_3',
 'lag_4',
 'lag_5',
 'lag_6',
 'lag_7',
 'lag_8',
 'lag_9',
 'lag_10',
 'lag_11',
 'lag_12',
 'presc_lag_1',
 'presc_lag_3',
 'presc_lag_6',
 'month',
 'trend',
 'prescribing_x_month']

In [78]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13578 entries, 12 to 16061
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   geography            13578 non-null  object        
 1   stratum              13578 non-null  object        
 2   date                 13578 non-null  datetime64[ns]
 3   organism             13578 non-null  object        
 4   resistance_pct       13578 non-null  float64       
 5   series_id            13578 non-null  object        
 6   model_region         13578 non-null  object        
 7   prescribing_rate     13578 non-null  float64       
 8   month                13578 non-null  int32         
 9   lag_1                13578 non-null  float64       
 10  lag_2                13578 non-null  float64       
 11  lag_3                13578 non-null  float64       
 12  lag_4                13578 non-null  float64       
 13  lag_5                13578 non-null

In [79]:
combined_df.shape

(13578, 26)

In [80]:
X_train = train_df[feature_cols]
y_train = train_df['resistance_pct']

In [81]:
X_test = test_df[feature_cols]
y_test = test_df['resistance_pct']

In [82]:
print(f"\nFeatures used ({len(feature_cols)}): {feature_cols[:3]}...{feature_cols[-2:]}")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")



Features used (18): ['lag_1', 'lag_2', 'lag_3']...['trend', 'prescribing_x_month']
X_train shape: (9162, 18)
X_test shape: (4416, 18)
y_train shape: (9162,)
y_test shape: (4416,)


Forecasting models

In [83]:
from sktime.forecasting.compose import make_reduction
from sktime.forecasting.base import ForecastingHorizon
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error


In [84]:
lr_forecaster = LinearRegression()

In [85]:
lgbm_forecaster =  LGBMRegressor(
        n_estimators=300,
        max_depth=5,
        learning_rate=0.05,
        num_leaves=31, #default
        subsample=0.8,
        colsample_bytree=0.8,
        min_child_samples=20,
        random_state=42, 
        verbose=-1
    )


In [86]:
xgb_forecaster = XGBRegressor(
        n_estimators = 300,
        max_depth=5,
        learning_rate = 0.05,
        subsample=0.8,
        colsample_bytree =0.8,
        random_state = 42,
        eval_metric = "rmse"
    )

In [87]:
rf_forecaster = RandomForestRegressor(
        n_estimators=300,
        max_depth=10,
        min_samples_split=5,
        min_samples_leaf=2,
        max_features='sqrt',
        random_state=42,
        n_jobs=-1
    )

Training models

In [88]:
models = {
    'LinearRegression' : lr_forecaster,
    'LightGBM': lgbm_forecaster,
    'XGBoost': xgb_forecaster,
    'Random Forest': rf_forecaster
}

In [89]:
for name, model in models.items():
    print(f"\nTraining {name}  ...")
    model.fit(X_train, y_train)
    print(f"done. {name} trained")


Training LinearRegression  ...
done. LinearRegression trained

Training LightGBM  ...
done. LightGBM trained

Training XGBoost  ...
done. XGBoost trained

Training Random Forest  ...
done. Random Forest trained


In [90]:
results= {}

In [91]:
from sklearn.metrics import mean_squared_error
import numpy as np
for name, model in models.items():
    print(f"\nPredicting with {name}")
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    results[name] = {'mae': mae, 'rmse':rmse}
    print(f"{name:20s} MAE: {mae} RMSE: {rmse}")


Predicting with LinearRegression
LinearRegression     MAE: 0.8845028818501726 RMSE: 1.3374225741614036

Predicting with LightGBM
LightGBM             MAE: 0.934112523552415 RMSE: 1.4310152413764616

Predicting with XGBoost
XGBoost              MAE: 0.9408481463205753 RMSE: 1.4425943616326729

Predicting with Random Forest
Random Forest        MAE: 0.9741137562314984 RMSE: 1.4956755255605738


In [92]:
from sklearn.metrics import mean_squared_error
import numpy as np
for name, model in models.items():
    print(f"\nPredicting with {name}")
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    results[name] = {'mae': mae, 'rmse':rmse}
    print(f"{name:20s} MAE: {mae} RMSE: {rmse}")


Predicting with LinearRegression
LinearRegression     MAE: 0.8845028818501726 RMSE: 1.3374225741614036

Predicting with LightGBM
LightGBM             MAE: 0.934112523552415 RMSE: 1.4310152413764616

Predicting with XGBoost
XGBoost              MAE: 0.9408481463205753 RMSE: 1.4425943616326729

Predicting with Random Forest
Random Forest        MAE: 0.9741137562314984 RMSE: 1.4956755255605738


In [93]:
# Evaluate on training
for name, model in models.items():
    y_pred_train = model.predict(X_train)
    mae = mean_absolute_error(y_train, y_pred_train)
    print(f"{name:20s} Train MAE: {mae:.3f}")


LinearRegression     Train MAE: 0.868
LightGBM             Train MAE: 0.686
XGBoost              Train MAE: 0.642
Random Forest        Train MAE: 0.691


In [94]:
def rolling_backtest (model, df, feature_cols, horizon=6, step=1):
    preds = []
    actuals = []
    dates = []

    all_dates = sorted(df["date"].unique())

    for i in range(24, len(all_dates) - horizon, step):
        train_end = all_dates[i]
        test_start = all_dates[i + 1]
        test_end = all_dates[i + horizon]

        train = df[df["date"] <= train_end]
        test = df[(df["date"] > train_end) & (df["date"] <= test_end)]

        X_train = train[feature_cols]
        y_train = train["resistance_pct"]
        X_test = test[feature_cols]
        y_test = test["resistance_pct"]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        preds.append(y_pred)
        actuals.append(y_test.values)
        dates.append(test["date"].values)
    return np.concatenate(preds), np.concatenate(actuals), np.concatenate(dates)

In [95]:
y_pred, y_true, pred_dates = rolling_backtest(
    xgb_forecaster,
    combined_df,
    feature_cols,
    horizon=6
)
print("MAE:", mean_absolute_error(y_true, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_true, y_pred)))

MAE: 0.9317998732044892
RMSE: 1.4381843953726186


In [96]:
final_model = xgb_forecaster

In [97]:
final_model.fit(
    combined_df[feature_cols],
    combined_df["resistance_pct"]
)

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.8
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'rmse'


In [98]:
def forecast(model, df, feature_cols, horizon=6, window_length=12):

    forecasts = []

    df = df.sort_values(["series_id", "date"]).copy()

    last_windows = (
        df.groupby("series_id", group_keys=False)
        .tail(window_length)
        .copy()
    )

    for sid, window in last_windows.groupby("series_id"):

        window = window.copy()
        preds = []

        last_date = window["date"].max()

        for h in range(1, horizon + 1):

            X = window[feature_cols].iloc[-1:]
            y_hat = model.predict(X)[0]

            forecast_date = last_date + pd.DateOffset(months=h)

            preds.append((forecast_date, y_hat))

            new_row = window.iloc[-1:].copy()
            new_row["resistance_pct"] = y_hat
            new_row["date"] = forecast_date

            for lag in range(window_length, 1, -1):
                new_row[f"lag_{lag}"] = new_row[f"lag_{lag-1}"]
            new_row["lag_1"] = y_hat

            if "trend" in feature_cols:
                new_row["trend"] = new_row["lag_1"] - new_row["lag_3"]

            if "month" in feature_cols:
                new_row["month"] = forecast_date.month

            if "prescribing_x_month" in feature_cols:
                new_row["prescribing_x_month"] = (
                    new_row["prescribing_rate"] * new_row["month"]
                )

            window = pd.concat([window, new_row], ignore_index=True)

        # CREATE FORECAST TABLE **AFTER FULL 6-MONTH LOOP**
        series_forecast = pd.DataFrame({
            "series_id": sid,
            "date": [p[0] for p in preds],
            "forecast": [p[1] for p in preds]
        })

        forecasts.append(series_forecast)

    forecast_df = pd.concat(forecasts, ignore_index=True)

    forecast_df[["organism","antibiotic","region"]] = (
        forecast_df["series_id"].str.split("__", expand=True)
    )

    forecast_df = forecast_df[
        ["organism","antibiotic","region","date","forecast"]
    ]

    return forecast_df


In [99]:
future_forecasts = forecast(
    final_model,
    combined_df,
    feature_cols
)

In [100]:
future_forecasts.head(50)

,organism,antibiotic,region,date,forecast
0,E-coli,Amikacin,East Midlands,2025-07-01,4.432085
1,E-coli,Amikacin,East Midlands,2025-08-01,4.059462
2,E-coli,Amikacin,East Midlands,2025-09-01,3.294566
3,E-coli,Amikacin,East Midlands,2025-10-01,3.303035
4,E-coli,Amikacin,East Midlands,2025-11-01,3.433069
5,E-coli,Amikacin,East Midlands,2025-12-01,3.678980
6,E-coli,Amikacin,East of England,2025-07-01,3.710473
7,E-coli,Amikacin,East of England,2025-08-01,3.420668
8,E-coli,Amikacin,East of England,2025-09-01,3.340389
9,E-coli,Amikacin,East of England,2025-10-01,3.405159


In [101]:
future_forecasts.shape


(1242, 5)

In [102]:
future_forecasts.to_csv("complete6months.csv")

In [103]:
combined_df['date']

12      2021-10-01
13      2021-11-01
14      2021-12-01
15      2022-01-01
16      2022-02-01
           ...    
16057   2025-02-01
16058   2025-03-01
16059   2025-04-01
16060   2025-05-01
16061   2025-06-01
Name: date, Length: 13578, dtype: datetime64[ns]

In [104]:
len(combined_df)

13578

Forecasting displayed on graph

In [105]:
forecast_df = pd.read_csv("complete6months.csv")

In [109]:
forecast_df.head()

,organism,antibiotic,region,date,forecast
0,E-coli,Amikacin,East Midlands,2025-07-01,4.432085
1,E-coli,Amikacin,East Midlands,2025-08-01,4.059462
2,E-coli,Amikacin,East Midlands,2025-09-01,3.294566
3,E-coli,Amikacin,East Midlands,2025-10-01,3.303034
4,E-coli,Amikacin,East Midlands,2025-11-01,3.433069


In [108]:
forecast_df = forecast_df.drop(columns=['Unnamed: 0'], errors='ignore')


In [107]:
combined_df.head()

,geography,stratum,date,organism,resistance_pct,series_id,model_region,prescribing_rate,month,lag_1,...,lag_8,lag_9,lag_10,lag_11,lag_12,presc_lag_1,presc_lag_3,presc_lag_6,trend,prescribing_x_month
12,East Midlands,Amikacin,2021-10-01,E-coli,1.52,E-coli__Amikacin__East Midlands,Midlands,1.34,10,1.85,...,0.17,0.53,0.93,0.92,0.53,1.31,1.28,1.25,0.21,13.40
13,East Midlands,Amikacin,2021-11-01,E-coli,1.17,E-coli__Amikacin__East Midlands,Midlands,1.37,11,1.52,...,0.94,0.17,0.53,0.93,0.92,1.34,1.30,1.25,-0.55,15.07
14,East Midlands,Amikacin,2021-12-01,E-coli,0.72,E-coli__Amikacin__East Midlands,Midlands,1.41,12,1.17,...,1.59,0.94,0.17,0.53,0.93,1.37,1.31,1.26,-0.68,16.92
15,East Midlands,Amikacin,2022-01-01,E-coli,1.90,E-coli__Amikacin__East Midlands,Midlands,1.41,1,0.72,...,1.65,1.59,0.94,0.17,0.53,1.41,1.34,1.28,-0.80,1.41
16,East Midlands,Amikacin,2022-02-01,E-coli,2.43,E-coli__Amikacin__East Midlands,Midlands,1.43,2,1.90,...,1.93,1.65,1.59,0.94,0.17,1.41,1.37,1.30,0.73,2.86


In [ ]:
%pip install plotly